REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [2]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [3]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [4]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


In [12]:
#check nulls
bus.isnull().sum()

business_id        0
name               0
address            0
city               0
state              0
postal_code      259
latitude        2716
longitude       2716
phone_number     619
dtype: int64

In [13]:
#check nulls
ins.isnull().sum()

business_id    0
score          0
date           0
type           0
dtype: int64

## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [16]:
#6315 including rows with nulls
bus.shape

(6315, 9)

In [17]:
bus.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'phone_number'],
      dtype='object')

#### Question 2b: How many unique business IDs are there?  

In [18]:
bus['business_id'].unique()

array([   10,    19,    24, ..., 90237, 90238, 90239])

In [19]:
#6315
bus['business_id'].count

<bound method Series.count of 0          10
1          19
2          24
3          31
4          45
        ...  
6310    90234
6311    90236
6312    90237
6313    90238
6314    90239
Name: business_id, Length: 6315, dtype: int64>

In [20]:
#googled this function to double check

number_of_unique_values = bus['business_id'].nunique()
print(number_of_unique_values)

6315


#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [23]:
name_counts = bus['name'].value_counts()
print(name_counts)

STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
                              ..
7-MISSION RESTAURANT           1
WONDERFUL DESSERT & CAFE       1
KITCHEN ISTANBAL               1
FAT BELLI DELI                 1
ALBA RAY'S                     1
Name: name, Length: 5692, dtype: int64


In [26]:
#googled .nlargest function; top 5 common businesses listed here

top_5_names = name_counts.nlargest(5)
print(top_5_names)

STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: name, dtype: int64


In [28]:
# Filter for businesses in San Francisco
sf_businesses = bus[bus['city'] == 'San Francisco']

In [30]:
sf_name_counts = bus['name'].value_counts()
print(sf_name_counts)

STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
                              ..
7-MISSION RESTAURANT           1
WONDERFUL DESSERT & CAFE       1
KITCHEN ISTANBAL               1
FAT BELLI DELI                 1
ALBA RAY'S                     1
Name: name, Length: 5692, dtype: int64


In [31]:
#top 5 common businesses in sf and how many
sf_top_5_names = sf_name_counts.nlargest(5)
print(sf_top_5_names)

STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: name, dtype: int64


## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [33]:
bus.dtypes

business_id       int64
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
phone_number     object
dtype: object

#### Question 3b

What are the unique values of postal_code?

In [34]:
num_of_unique_zip_values = bus['postal_code'].nunique()
print(num_of_unique_zip_values)

46


#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [35]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [36]:
vb = bus[bus['postal_code'].isin(validZip)]

In [37]:
# 5999 total businesses

vb.shape

(5999, 9)

In [42]:
#5438 unique businesses, Starbucks as 1 business

vb_name_counts = vb['name'].value_counts()
print(vb_name_counts)

STARBUCKS COFFEE                   71
PEET'S COFFEE & TEA                24
WALGREENS                          11
SAN FRANCISCO SOUP COMPANY         11
LEE'S DELI                         10
                                   ..
NORDSTROM ESPRESSO BAR              1
THE MARSH CAFE                      1
ONO HAWAIIN BBQ                     1
SAN FRANCISCO PUBLIC MONTESSORI     1
ALBA RAY'S                          1
Name: name, Length: 5438, dtype: int64


In [44]:
# double checking above
vb['name'].nunique()

5438

## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [45]:
#2483

vb[pd.isnull(vb['longitude'])]

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
200,821,LA SANTANECA,3781 MISSION ST,San Francisco,CA,94110,NaN,NaN,+14156481034
210,852,NEW HOLLYWOOD BAKERY & RESTAURANT,652 PACIFIC AVE,San Francisco,CA,94133,NaN,NaN,+14153979919
760,2934,LOU'S FISH SHACK,300 JEFFERSON ST,San Francisco,CA,94133,NaN,NaN,+14157715687
826,3123,LES JOULINS,44 ELLIS ST,San Francisco,CA,94102,NaN,NaN,+14153975397
1100,4323,DIVA'S,1081 POST ST,San Francisco,CA,94109,NaN,NaN,+14154743482
...,...,...,...,...,...,...,...,...,...
6310,90234,ACCESS SFUSD THE ARC,1520 HOWARD ST,San Francisco,CA,94103,NaN,NaN,NaN
6311,90236,PHO DONG HUONG,667 MONTEREY BLVD,San Francisco,CA,94127,NaN,NaN,+14154945048
6312,90237,WIN'S RESTAURANT,3040 TARAVAL ST,San Francisco,CA,94116,NaN,NaN,+14158606036
6313,90238,USA POWER MARKET,1524 OCEAN AVE,San Francisco,CA,94112,NaN,NaN,+14153331878


In [46]:
missing_longitude_count = vb['longitude'].isnull().sum()
print("Number of businesses missing longitude values:", missing_longitude_count)

Number of businesses missing longitude values: 2483


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [47]:
grouped = vb.groupby('postal_code')['longitude'].agg(null_lon=lambda x: x.isnull().sum(), not_null_lon=lambda x: x.notnull().sum()).reset_index()

# Create the new dataframe
new_df = pd.DataFrame({
    'postal_code': grouped['postal_code'],
    'null_lon': grouped['null_lon'],
    'not_null_lon': grouped['not_null_lon']
})

print(new_df)

   postal_code  null_lon  not_null_lon
0        94102       196           262
1        94103       252           306
2        94104        59            74
3        94105       100           126
4        94107       247           204
5        94108        82           142
6        94109       146           237
7        94110       251           334
8        94111       115           167
9        94112        59           131
10       94114       108           122
11       94115        76           162
12       94116        35            58
13       94117        67           130
14       94118       100           143
15       94121        55           106
16       94122       123           147
17       94123        53           116
18       94124        88            76
19       94127        28            43
20       94131        14            34
21       94132        65            69
22       94133       135           282
23       94134        29            45


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [48]:
# total number of businesses per zip
total_businesses = grouped['null_lon'] + grouped['not_null_lon']
print(total_businesses)

0     458
1     558
2     133
3     226
4     451
5     224
6     383
7     585
8     282
9     190
10    230
11    238
12     93
13    197
14    243
15    161
16    270
17    169
18    164
19     71
20     48
21    134
22    417
23     74
dtype: int64


In [65]:
# find proportion 
grouped['proportion_missing'] = grouped['null_lon'] / total_businesses
print(grouped['proportion_missing'])

0     0.427948
1     0.451613
2     0.443609
3     0.442478
4     0.547672
5     0.366071
6     0.381201
7     0.429060
8     0.407801
9     0.310526
10    0.469565
11    0.319328
12    0.376344
13    0.340102
14    0.411523
15    0.341615
16    0.455556
17    0.313609
18    0.536585
19    0.394366
20    0.291667
21    0.485075
22    0.323741
23    0.391892
Name: proportion_missing, dtype: float64


In [66]:
# sort group

sorted_df = grouped.sort_values(by='proportion_missing', ascending=False)

In [67]:
print(sorted_df.head(5)[['postal_code', 'proportion_missing']])

   postal_code  proportion_missing
4        94107            0.547672
18       94124            0.536585
21       94132            0.485075
10       94114            0.469565
16       94122            0.455556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [68]:
# 15430
ins.shape

(15430, 4)

In [69]:
# only 5730 unique businesses

ins['business_id'].nunique()

5730

#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [74]:
type_counts = ins['type'].value_counts()
print(type_counts)

routine      15429
complaint        1
Name: type, dtype: int64


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [78]:
ins.dtypes

business_id     int64
score           int64
date            int64
type           object
dtype: object

In [79]:
#convert 'date' to string
ins['date'] = ins['date'].astype(str)

#extract first 4 characters from date column
ins['year'] = ins['date'].str[:4]

print(ins)

       business_id  score      date     type  year
0               10     82  20160503  routine  2016
1               10     94  20140729  routine  2014
2               10     92  20140114  routine  2014
3               19     94  20160513  routine  2016
4               19     94  20141110  routine  2014
...            ...    ...       ...      ...   ...
15425        89515     98  20161103  routine  2016
15426        89701    100  20161206  routine  2016
15427        89790     96  20161129  routine  2016
15428        89803    100  20161215  routine  2016
15429        89900     93  20161206  routine  2016

[15430 rows x 5 columns]


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [82]:
# Get the minimum and maximum values in  'year' 
min_year = ins['year'].min()
max_year = ins['year'].max()

print(min_year,"-", max_year)

2013 - 2016


In [84]:
# Calculate the number of inspections for each year
inspections_per_year = ins['year'].value_counts().sort_index()

# Calculate the mean number of inspections per year
mean_inspections_per_year = inspections_per_year.mean()

# Print the number of inspections per year
print("Number of inspections per year:")
print(inspections_per_year)

Number of inspections per year:
2013      38
2014    5629
2015    3923
2016    5840
Name: year, dtype: int64


In [88]:
# Print the mean number of inspections per year
print("Mean number of inspections per year:", mean_inspections_per_year)

Mean number of inspections per year: 3857.5


In [87]:
# Filtering 
inspections_per_year = inspections_per_year[inspections_per_year >= 50]

# Create a new df named ins3 containing the filtered data
ins3 = ins[ins['year'].isin(inspections_per_year.index)]

# Print the new ins3
print(ins3)

       business_id  score      date     type  year
0               10     82  20160503  routine  2016
1               10     94  20140729  routine  2014
2               10     92  20140114  routine  2014
3               19     94  20160513  routine  2016
4               19     94  20141110  routine  2014
...            ...    ...       ...      ...   ...
15425        89515     98  20161103  routine  2016
15426        89701    100  20161206  routine  2016
15427        89790     96  20161129  routine  2016
15428        89803    100  20161215  routine  2016
15429        89900     93  20161206  routine  2016

[15392 rows x 5 columns]


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [93]:
ins_2016 = ins[ins['year'] == '2016']
print(ins_2016)

       business_id  score      date     type  year
0               10     82  20160503  routine  2016
3               19     94  20160513  routine  2016
6               24     98  20161005  routine  2016
7               24     96  20160311  routine  2016
10              31     84  20160816  routine  2016
...            ...    ...       ...      ...   ...
15425        89515     98  20161103  routine  2016
15426        89701    100  20161206  routine  2016
15427        89790     96  20161129  routine  2016
15428        89803    100  20161215  routine  2016
15429        89900     93  20161206  routine  2016

[5840 rows x 5 columns]


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [97]:
# left join on business_id
merged_df = pd.merge(bus, ins_2016, on='business_id', how='left')

In [98]:
merged_df.head(7)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816,routine,2016
5,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,84.0,20160614,routine,2016
6,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,78.0,20160104,routine,2016


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [102]:
# rank them
lowest_rated = merged_df.sort_values(by='score').head(20)

# Print the names, addresses, and ratings of the 20 lowest rated businesses
print("20 lowest rated businesses:")
print(lowest_rated[['name', 'address', 'score']])


20 lowest rated businesses:
                             name              address  score
4892      GOLDEN RIVER RESTAURANT     5827 GEARY BLVD    52.0
7325                    POKI TIME     2101 LOMBARD ST    55.0
6281                 CRAZY PEPPER   2257 SAN JOSE AVE    55.0
4873                   UNCLE CAFE       65 WAVERLY PL    55.0
3008             HING WANG BAKERY        339 JUDAH ST    56.0
5627                   GOLDEN WOK   295 B ORIZABA AVE    56.0
5383                   TOP'S CAFE  2460 SAN BRUNO AVE    57.0
6067    L & G VIETNAMESE SANDWICH         602 EDDY ST    57.0
836                  RED A BAKERY      634 CLEMENT ST    57.0
4209      SAN TUNG RESTAURANT LLC      1031 IRVING ST    57.0
549           MA'S DIM SUM & CAFE      1315 POWELL ST    57.0
4903  NEW GARDEN RESTAURANT, INC.       716 KEARNY ST    57.0
6645    KOBANI MEDITERANEAN GRILL       3242 SCOTT ST    58.0
624           JOANIE'S DINER INC.   1329 COLUMBUS AVE    59.0
6938         PAK NAM THAI CUISINE       65

## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!